In [ ]:
from ALLCools.mcds import MCDS
from ALLCools.plot import *
from ALLCools.integration import confusion_matrix_clustering

from wmb import cemba, aibs, brain

import pandas as pd
import numpy as np
import anndata
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
category_key = 'L2'
plot_key = 'L2'

In [ ]:
mc_annot = cemba.get_mc_annot()
rna_annot = aibs.get_smart_annot()

In [ ]:
adata_merge = anndata.read_h5ad('final_with_coords.h5ad')

In [ ]:
adata_merge

In [ ]:
rna_adata = adata_merge[adata_merge.obs['Modality'] == 'RNA'].copy()
mc_adata = adata_merge[adata_merge.obs['Modality'] == 'mC'].copy()

In [ ]:
rna_meta = adata_merge.obs[adata_merge.obs['Modality'] == 'RNA'].copy()
mc_meta = adata_merge.obs[adata_merge.obs['Modality'] == 'mC'].copy()

## Determine integration group
In RNA L2 resolution, assign RNA L2 to its most probable mC clusters where the sum of probability > 0.95

In [ ]:
confusion_matrix = pd.read_hdf(f'{category_key}.overlap_score.hdf')

In [ ]:
row_group, col_group, confusion_matrix = confusion_matrix_clustering(
    confusion_matrix, min_value=0, max_value=0.9)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 7), dpi=300)
sns.heatmap(confusion_matrix, ax=ax, vmin=0, vmax=0.5)

In [ ]:
rna_adata.obs['InteGroup'] = rna_adata.obs[category_key].map(row_group)
rna_adata.obs['InteGroup'].value_counts()

In [ ]:
mc_adata.obs['InteGroup'] = mc_adata.obs[category_key].map(col_group)
mc_adata.obs['InteGroup'].value_counts()

## Manual Adjust

### Merge Integration Group

In [ ]:
inte_group_map = {
    # if need to merge integration group, add k:v here
    # 1: 0,
}

rna_adata.obs['InteGroup'] = rna_adata.obs['InteGroup'].map(
    lambda i: inte_group_map[i] if i in inte_group_map else i)
mc_adata.obs['InteGroup'] = mc_adata.obs['InteGroup'].map(
    lambda i: inte_group_map[i] if i in inte_group_map else i)

### Plot integration groups

In [ ]:
from ALLCools.plot.color import level_one_palette

inte_group_palette = level_one_palette(
    pd.concat([rna_adata.obs['InteGroup'], mc_adata.obs['InteGroup']]), 
    palette='tab20'
)

In [ ]:
fig, axes = plt.subplots(figsize=(10, 10), ncols=2, nrows=2, dpi=300)

ax = axes[0, 0]
categorical_scatter(ax=ax,
                    data=rna_adata,
                    coord_base='tsne',
                    palette='tab20',
                    hue=plot_key,
                    text_anno=plot_key,
                    max_points=None)
ax.set(title=f'RNA {plot_key}')

ax = axes[1, 0]
categorical_scatter(ax=ax,
                    data=rna_adata,
                    coord_base='tsne',
                    hue='InteGroup',
                    text_anno='InteGroup',
                    palette=inte_group_palette,
                    max_points=None)
ax.set(title='RNA Inte. Group')

ax = axes[0, 1]
categorical_scatter(ax=ax,
                    data=mc_adata,
                    coord_base='tsne',
                    palette='tab20',
                    hue=plot_key,
                    text_anno=plot_key,
                    max_points=None)
ax.set(title=f'mC {plot_key}')
ax = axes[1, 1]
categorical_scatter(ax=ax,
                    data=mc_adata,
                    coord_base='tsne',
                    hue='InteGroup',
                    text_anno='InteGroup',
                    palette=inte_group_palette,
                    max_points=None)
ax.set(title='RNA Inte. Group')

## Save Integration Group

In [ ]:
# map integration group to all cells based on intra-dataset clustering
counts = mc_adata.obs.groupby(category_key)['InteGroup'].value_counts()
mc_cluster_to_inte_group = {
    mc: inte_group
    for mc, inte_group in counts[counts > 0].index
}
mc_cell_inte_group = mc_annot[category_key].to_pandas().map(
    mc_cluster_to_inte_group).dropna().astype(int)

mc_cell_inte_group.to_csv('mc_integration_group.csv.gz')
mc_cell_inte_group.value_counts()

In [ ]:
counts = rna_adata.obs.groupby(category_key)['InteGroup'].value_counts()
rna_cluster_to_inte_group = {
    rna: inte_group
    for rna, inte_group in counts[counts > 0].index
}
rna_cell_inte_group = rna_annot[category_key].to_pandas().map(
    rna_cluster_to_inte_group).dropna().astype(int)

rna_cell_inte_group.to_csv('rna_integration_group.csv.gz')
rna_cell_inte_group.value_counts()

## Plot Individual Group

1. is there mC cluster do not match to any RNA cluster?
2. different separatetion between mC / RNA

In [ ]:
def plot_single_group(group):
    rna_hue = rna_adata.obs['InteGroup'] == group
    mc_hue = mc_adata.obs['InteGroup'] == group

    fig, axes = plt.subplots(figsize=(10, 10), ncols=2, nrows=2, dpi=300)

    ax = axes[0, 0]
    categorical_scatter(ax=ax,
                        data=rna_adata,
                        coord_base='tsne',
                        hue='L1_annot',
                        text_anno='L1_annot',
                        max_points=None)
    ax.set(title='RNA L1 Annot')

    ax = axes[1, 0]
    categorical_scatter(ax=ax,
                        data=rna_adata,
                        coord_base='tsne',
                        hue=rna_hue,
                        palette={
                            True: 'red',
                            False: 'lightgrey'
                        },
                        text_anno='InteGroup',
                        max_points=None)
    ax.set(title='RNA Inte. Group')

    ax = axes[0, 1]
    categorical_scatter(ax=ax,
                        data=mc_adata,
                        coord_base='tsne',
                        hue='L1_annot',
                        text_anno='L1_annot',
                        max_points=None)
    ax.set(title='mC L1 Annot')

    ax = axes[1, 1]
    categorical_scatter(ax=ax,
                        data=mc_adata,
                        coord_base='tsne',
                        hue=mc_hue,
                        palette={
                            True: 'red',
                            False: 'lightgrey'
                        },
                        text_anno='InteGroup',
                        max_points=None)
    ax.set(title='RNA Inte. Group')
    return fig

In [ ]:
# import matplotlib.backends.backend_pdf
# 
# with matplotlib.backends.backend_pdf.PdfPages("integration_groups.pdf") as pdf:
#     for group in rna_adata.obs['InteGroup'].unique():
#         fig = plot_single_group(group)
#         pdf.savefig(fig)